In [62]:
#import libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from requests import get
import json

import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import asyncio
import concurrent.futures
import re
import os
import nltk
from textblob import Word
from nltk.tag import pos_tag
from nltk.corpus import reuters
from nltk.corpus import stopwords
from nltk import download as nltk_download
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split
from sklearn.datasets.twenty_newsgroups import fetch_20newsgroups

stemmer = SnowballStemmer("english")
stop_words=stopwords.words('english')

session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)




## Yale Climate Databse -- skip this section

In [35]:
#load excel with list of climate change policy docs

climate_sites1 = pd.read_csv('climate_data_1.csv', encoding = "ISO-8859-1")

In [44]:
climate_sites1[:8]

,Title,Policy[Source or references],Policy[Date of decision],Policy[Implementation state],Policy[Policy type]
0,"""All inclusive"" Feed-in Tariff for small renew...",http://www.gse.it/attivita/Incentivazioni%20Fo...,2008,Superseded,Renewables
1,"""Energy of the Future"" monitoring process Germ...",http://www.bundesnetzagentur.de/EN/Areas/Energ...,2011,Implemented,"Changing activity, Energy efficiency, Renewabl..."
2,"""Energy-efficient Systems in Trade and Industr...",http://www.stromeffizienz.de/,0,Ended,Energy efficiency
3,"""Kids ISO 14000 Program"" for Energy Conservati...",http://www.iso.org/iso/en/kidsiso14000/index.html,2005,Implemented,Energy efficiency
4,"""Old"" Feed-In Premium for photovoltaic systems...",http://www.gse.it/attivita/ContoEnergiaF/servi...,2005,Superseded,Renewables
5,10 Star NatHERS Rating for Buildings Australia...,http://www.nathers.gov.au/governance;\r\rhttp:...,2015,Implemented,Energy efficiency
6,10 Year Transport Plan United Kingdom 2000,http://webarchive.nationalarchives.gov.uk/+/ht...,2000,Superseded,"Changing activity, Nuclear or CCS or fuel switch"
7,10-year Strategy for Fuel Switching to Natural...,http://www.iea.org/policiesandmeasures/pams/au...,2006,Ended,Nuclear or CCS or fuel switch


In [37]:
links = climate_sites1['Policy[Source or references]']

In [45]:
test = links[7]
try:
    response = get(test)
    
except:
    print("bad link")

else:
    
    if "page not found" not in response.text.lower():
        print(test)
        #print(response.text)
    


http://www.iea.org/policiesandmeasures/pams/australia/name-23012-en.php?s=dHlwZT1jYyZzdGF0dXM9T2s,&return=PGRpdiBjbGFzcz0ic3ViTWVudSI-PGRpdiBjbGFzcz0iYnJlYWRjcnVtYnMiPjxhIGhyZWY9Ii8iPkludGVybmF0aW9uYWwgRW5lcmd5IEFnZW5jeSZ6d25qOzwvYT4mbmJzcDsmZ3Q7Jm5ic3A7PGEgaHJlZj0iL3BvbGljaWVzYW5kbWVhc3VyZXMvIj5Qb2xpY2llcyBhbmQgTWVhc3VyZXM8L2E-Jm5ic3A7Jmd0OzxhIGhyZWY9Ii9wb2xpY2llc2FuZG1lYXN1cmVzL2NsaW1hdGVjaGFuZ2UvaW5kZXgucGhwIj4mbmJzcDtDbGltYXRlIENoYW5nZTwvYT4mbmJzcDsmZ3Q7Jm5ic3A7U2VhcmNoIFJlc3VsdDwvZGl2Pg,,


## IEA Data

In [12]:
get_countries = "https://www.iea.org/policiesandmeasures/climatechange/"
g_c = get(get_countries)

soup_c = BeautifulSoup(g_c.text, 'html.parser')
c_list = soup_c.find("ul", class_ ="listexpander")
c_loop = c_list.find_all("label")

country_store = []
for c in c_loop:
    country_store.append(c.text)
    
full_list = country_store[4:]


In [13]:
#for climate policies and measure database
policy_link_store = []
for country in full_list:
    if " " in country:
        amend = country.replace(" ", "%20")
        policy_link = "https://www.iea.org/policiesandmeasures/climatechange/?country="+amend 
        policy_link_store.append(policy_link)
    else:
        policy_link = "https://www.iea.org/policiesandmeasures/climatechange/?country="+country
        policy_link_store.append(policy_link)
        



In [31]:
#for renewable energy
renewables_store = []
for country in full_list:
    if " " in country:
        amend = country.replace(" ", "%20")
        policy_link = "https://www.iea.org/policiesandmeasures/renewableenergy/?country="+amend 
        renewables_store.append(policy_link)
    else:
        policy_link = "https://www.iea.org/policiesandmeasures/renewableenergy/?country="+country
        renewables_store.append(policy_link)
        



In [37]:
#energy efficiency

efficiency_store = []
for country in full_list:
    if " " in country:
        amend = country.replace(" ", "%20")
        policy_link = "https://www.iea.org/policiesandmeasures/energyefficiency/?country="+amend 
        efficiency_store.append(policy_link)
    else:
        policy_link = "https://www.iea.org/policiesandmeasures/energyefficiency/?country="+country
        efficiency_store.append(policy_link)
        

In [38]:
## where each country has a table of all the policies and measures
print(len(policy_link_store))
print(len(renewables_store))
print(len(efficiency_store))

59
51
51


In [16]:
#get individual links for each policy and measure page for each country 
page_link_store = []

for p in policy_link_store:
    g_p = session.get(p)
    soup_p = BeautifulSoup(g_p.text, 'html.parser')
    entries = soup_p.find_all("tr")[1:]
    for entry in entries:
        ext = entry.find("a", href = True)
        link = "https://www.iea.org/" + ext._attr_value_as_string("href")
        page_link_store.append(link)

In [33]:
#get individual links for each renewable energy policy for each country 
renewpage_link_store = []

for p in renewables_store:
    g_p = session.get(p)
    soup_p = BeautifulSoup(g_p.text, 'html.parser')
    entries = soup_p.find_all("tr")[1:]
    for entry in entries:
        ext = entry.find("a", href = True)
        link = "https://www.iea.org/" + ext._attr_value_as_string("href")
        renewpage_link_store.append(link)

In [41]:
#get individual links for each energy efficiency policy for each country 
eff_page_link_store = []

for p in efficiency_store:
    g_p = session.get(p)
    soup_p = BeautifulSoup(g_p.text, 'html.parser')
    entries = soup_p.find_all("tr")[1:]
    for entry in entries:
        ext = entry.find("a", href = True)
        link = "https://www.iea.org/" + ext._attr_value_as_string("href")
        eff_page_link_store.append(link)

In [42]:
print(len(page_link_store))
print(len(renewpage_link_store))
print(len(eff_page_link_store))

2790
1492
1995


In [51]:
#climate policy
full_set = []

async def main():
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:

        loop = asyncio.get_event_loop()
        futures = [
            loop.run_in_executor(
                executor, 
                session.get, 
                link
            )
            for link in page_link_store
        ]
        
        for response in await asyncio.gather(*futures):
            #entry = dict()
            soup_e = BeautifulSoup(response.text, 'html.parser')

            for script in soup_e(["script", "style"]):
                script.extract()

            text = soup_e.get_text()
            
                #entry["url"] = link
                #entry["text"] = text
            
            full_set.append(text)


loop = asyncio.get_event_loop()
loop.run_until_complete(main())

In [54]:
#renewable energy policy
full_renew_set = []

async def main():
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:

        loop = asyncio.get_event_loop()
        futures = [
            loop.run_in_executor(
                executor, 
                session.get, 
                link
            )
            for link in renewpage_link_store
        ]
        
        for response in await asyncio.gather(*futures):
            entry = dict()
            soup_e = BeautifulSoup(response.text, 'html.parser')
            
            for script in soup_e(["script", "style"]):
                script.extract()

            text = soup_e.get_text()
            
                #entry["url"] = link
                #entry["text"] = text
            
            full_renew_set.append(text)


loop = asyncio.get_event_loop()
loop.run_until_complete(main())

In [56]:
# energy efficiency policy
full_eff_set = []

async def main():
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:

        loop = asyncio.get_event_loop()
        futures = [
            loop.run_in_executor(
                executor, 
                session.get, 
                link
            )
            for link in eff_page_link_store
        ]
        
        for response in await asyncio.gather(*futures):
            entry = dict()
            soup_e = BeautifulSoup(response.text, 'html.parser')

            for script in soup_e(["script", "style"]):
                script.extract()

            text = soup_e.get_text()
            
                #entry["url"] = link
                #entry["text"] = text
            
            full_eff_set.append(text)


loop = asyncio.get_event_loop()
loop.run_until_complete(main())

In [65]:
print('climate change policy db:', len(full_set))
print('renewable energy policy db:', len(full_renew_set))
print('energy efficiency db:', len(full_eff_set))
complete = full_set + full_renew_set + full_eff_set
print('total:', len(complete))

climate change policy db: 2790
renewable energy policy db: 1492
energy efficiency db: 1995
total: 6277


In [63]:
ieadb = {'text': complete}
with open('ieapandm.json', 'w') as outfile:
    json.dump(ieadb, outfile)

In [64]:
load = pd.read_json('ieapandm.json', 'column')
load['label'] = 1
positive_df = load[['text', 'label']]
positive_df

,text,label
0,\n\n\n\n\nIEA - Albania\n\n\n\n\n\n\n\nInterna...,1
1,\n\n\n\n\nIEA - Albania\n\n\n\n\n\n\n\nInterna...,1
2,\n\n\n\n\nIEA - Australia\n\n\n\n\n\n\n\nInter...,1
3,\n\n\n\n\nIEA - Australia\n\n\n\n\n\n\n\nInter...,1
4,\n\n\n\n\nIEA - Australia\n\n\n\n\n\n\n\nInter...,1
5,\n\n\n\n\nIEA - Australia\n\n\n\n\n\n\n\nInter...,1
6,\n\n\n\n\nIEA - Australia\n\n\n\n\n\n\n\nInter...,1
7,\n\n\n\n\nIEA - Australia\n\n\n\n\n\n\n\nInter...,1
8,\n\n\n\n\nIEA - Australia\n\n\n\n\n\n\n\nInter...,1
9,\n\n\n\n\nIEA - Australia\n\n\n\n\n\n\n\nInter...,1


In [12]:
in_domain = look['text'].tolist()

## Get News20 data

In [4]:
categories = ['alt.atheism', 'comp.graphics',
              'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware',
              'comp.windows.x', 'misc.forsale', 'rec.autos',
              'rec.motorcycles', 'rec.sport.baseball',
              'rec.sport.hockey', 'sci.crypt', 'sci.electronics',
              'sci.med', 'sci.space', 'soc.religion.christian',
              'talk.politics.guns', 'talk.politics.mideast',
              'talk.politics.misc', 'talk.religion.misc']
newsgroups = fetch_20newsgroups(categories=categories)
    


In [7]:
len(newsgroups['data'])

10723

## Combine Datasets

In [13]:
nn_data = {'in': in_domain, 'out': newsgroups['data'] }

In [15]:
import json
with open('data.json', 'w') as outfile:
    json.dump(nn_data, outfile)